In [1]:
from pyfiles import (parametrics as par,
    complex as cplx,
    matrix as mat,
    vineyard as vin,
    input as inp,
    plot as ourplot,
    grid as grid,
    utils as utils,
    examples as ex,
    sneaky_matrix as sneaky
)
from pyfiles.examples import CameraOpt

# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2
%env RUST_BACKTRACE=1

import numpy as np
from copy import deepcopy

from typing import *

import matplotlib.pyplot as plt
%matplotlib inline

import mars

env: RUST_BACKTRACE=1


In [4]:
filename = 'input/hinge.obj'

class vineyard2(vin.vineyard):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.swaps = []

  def get_point_key(self, point: np.ndarray):
    return None
  
  def prune(self, s1: cplx.simplex, s2: cplx.simplex, old_state, new_state):
    if s1.dim() == s2.dim():
      self.swaps.append((s1, s2, old_state, new_state))
    return True

p0 = [0.3, 0.3, 0.3]
p1 = [0.6, 0.3, 0.3]


py_complex = inp.read_obj(filename)
py_vineyard = vineyard2(py_complex)
py_state0 = py_vineyard.reduce(p0)
py_state1 = py_vineyard.reduce_vine(py_state0, p1)

print('Python swaps')
print(f'#swaps = {len(py_vineyard.swaps)}')
print('   '.join([f'{a.prettyrepr()}-{b.prettyrepr()}' for (a, b, _, _) in py_vineyard.swaps]))


rs_complex = mars.read_from_obj(filename)
rs_state0 = mars.reduce_from_scratch(rs_complex, p0)
swaps = mars.vineyards_123(rs_complex, rs_state0, p1)

sdict = {0: 'v', 1: 'e', 2: 'f'}
print()
print('Rust swaps') 
print(f'#swaps = {len(swaps)}')
print('   '.join([f'{sdict[dim]}{a}-{sdict[dim]}{b}' for (dim, (a, b)) in swaps]))


other_order
[0, 1, 4, 5, 6, 8, 9, 11, 2, 7, 3, 10]
[simplex ∅ bd [], simplex v0 bd [-1], simplex v1 bd [-1], simplex v3 bd [-1], simplex e0 bd [0, 1], simplex e3 bd [1, 3], simplex e4 bd [3, 0], simplex t1 bd [0, 3, 4], simplex v2 bd [-1], simplex e1 bd [1, 2], simplex e2 bd [2, 0], simplex t0 bd [0, 1, 2]]
Python swaps
#swaps = 1
e2-e1
Distances in compute_permutations: (key_point=[0.3, 0.3, 0.3])
[FloatOrd(0.6699999999999999), FloatOrd(0.6699999999999999), FloatOrd(1.0699999999999998), FloatOrd(0.6699999999999999)]
[FloatOrd(0.6699999999999999), FloatOrd(1.0699999999999998), FloatOrd(1.0699999999999998), FloatOrd(0.6699999999999999), FloatOrd(0.6699999999999999)]
[FloatOrd(1.0699999999999998), FloatOrd(0.6699999999999999)]

Rust swaps
#swaps = 2
e3-e4   e0-e4


[src/lib.rs:308] &vine_ordering0 = Permutation {
    forwards: [
        0,
        3,
        1,
        2,
    ],
    backwards: [
        0,
        2,
        3,
        1,
    ],
}
[src/lib.rs:309] &vine_ordering1 = Permutation {
    forwards: [
        4,
        0,
        3,
        1,
        2,
    ],
    backwards: [
        1,
        3,
        4,
        2,
        0,
    ],
}
[src/lib.rs:310] &vine_ordering2 = Permutation {
    forwards: [
        1,
        0,
    ],
    backwards: [
        1,
        0,
    ],
}


Distances in compute_permutations: (key_point=[0.6, 0.3, 0.3])
[FloatOrd(0.33999999999999997), FloatOrd(0.9399999999999998), FloatOrd(0.7399999999999999), FloatOrd(0.94)]
[FloatOrd(0.9399999999999998), FloatOrd(0.9399999999999998), FloatOrd(0.7399999999999999), FloatOrd(0.94), FloatOrd(0.94)]
[FloatOrd(0.9399999999999998), FloatOrd(0.94)]
vineyards_123


In [3]:
print(rs_complex.simplices_per_dim[1][0])
print(rs_complex.simplices_per_dim[1][1])

# e0 = [0, 1]
# e1 = [1, 2]

# first point:
# v0 0.669999
# v1 0.669999
# v2 1.069999
# v3 0.669999

# e0 = 0.6699
# e1 = 1.0699

# second point:
# v0 0.429999
# v1 0.83
# v2 0.83
# v3 0.829999

# e0 = 0.83
# e1 = 0.83

e(0; bnd=[0, 1])
e(1; bnd=[1, 2])
